In [63]:
import pandas as pd

In [64]:
df=pd.DataFrame()
base_url="https://storage.googleapis.com/nyc-taxi-data-covid/data/"
for i in range(2019,2022):
    for j in range(1,13):
        url=base_url+str(i)+"/yellow_tripdata_"+str(i)+"-"+f"{j:02}"+".parquet"
        df_new=pd.read_parquet(url)
        df=pd.concat([df_new])
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2021-12-01 00:19:51,2021-12-01 00:37:01,1.0,9.30,1.0,N,138,141,1,26.5,4.25,0.5,7.60,6.55,0.3,45.70,2.5,1.25
1,2,2021-12-01 00:29:07,2021-12-01 00:45:13,2.0,2.76,1.0,N,238,42,2,13.0,0.50,0.5,0.00,0.00,0.3,16.80,2.5,0.00
2,1,2021-12-01 00:12:40,2021-12-01 00:27:17,0.0,3.40,1.0,N,239,74,1,13.5,3.00,0.5,2.00,0.00,0.3,19.30,2.5,0.00
3,1,2021-12-01 00:10:18,2021-12-01 00:19:20,1.0,1.30,1.0,N,148,87,1,6.5,3.00,0.5,2.05,0.00,0.3,12.35,2.5,0.00
4,1,2021-12-01 00:25:12,2021-12-01 00:39:07,1.0,3.10,1.0,N,231,246,1,12.5,3.00,0.5,2.50,0.00,0.3,18.80,2.5,0.00


In [65]:
df.memory_usage(index=True).sum()

488584216

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214369 entries, 0 to 3214368
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [67]:
df['tpep_pickup_datetime']=pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime']=pd.to_datetime(df['tpep_dropoff_datetime'])

In [68]:
df = df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.index

In [69]:
datetime_dim = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)
datetime_dim['tpep_pickup_datetime'] = datetime_dim['tpep_pickup_datetime']
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

datetime_dim['tpep_dropoff_datetime'] = datetime_dim['tpep_dropoff_datetime']
datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday


datetime_dim['datetime_id'] = datetime_dim.index

# datetime_dim = datetime_dim.rename(columns={'tpep_pickup_datetime': 'datetime_id'}).reset_index(drop=True)
datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]
#
datetime_dim.head()

,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2021-12-01 00:19:51,0,1,12,2021,2,2021-12-01 00:37:01,0,1,12,2021,2
1,1,2021-12-01 00:29:07,0,1,12,2021,2,2021-12-01 00:45:13,0,1,12,2021,2
2,2,2021-12-01 00:12:40,0,1,12,2021,2,2021-12-01 00:27:17,0,1,12,2021,2
3,3,2021-12-01 00:10:18,0,1,12,2021,2,2021-12-01 00:19:20,0,1,12,2021,2
4,4,2021-12-01 00:25:12,0,1,12,2021,2,2021-12-01 00:39:07,0,1,12,2021,2


In [70]:
passenger_count_dim = df[['passenger_count']].reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[['passenger_count_id','passenger_count']]

trip_distance_dim = df[['trip_distance']].reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[['trip_distance_id','trip_distance']]

In [71]:
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

rate_code_dim = df[['RatecodeID']].reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
rate_code_dim = rate_code_dim[['rate_code_id','RatecodeID','rate_code_name']]


In [72]:
zone_lookup=pd.read_csv("data/taxi_zone_lookup.csv")
pickup_location_dim=df[['PULocationID']].reset_index(drop=True)
pickup_location_dim['pickup_location_id'] = pickup_location_dim.index
pickup_location_dim=pickup_location_dim.merge(zone_lookup,left_on='PULocationID',right_on='LocationID')
pickup_location_dim['pickup_borough']=pickup_location_dim['Borough']
pickup_location_dim['pickup_zone']=pickup_location_dim['Zone']
pickup_location_dim=pickup_location_dim[['pickup_location_id','pickup_borough','pickup_zone']]
pickup_location_dim.head()

,pickup_location_id,pickup_borough,pickup_zone
0,0,Queens,LaGuardia Airport
1,8,Queens,LaGuardia Airport
2,10,Queens,LaGuardia Airport
3,20,Queens,LaGuardia Airport
4,32,Queens,LaGuardia Airport


In [73]:
dropoff_location_dim=df[['DOLocationID']].reset_index(drop=True)
dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index
dropoff_location_dim=dropoff_location_dim.merge(zone_lookup,left_on='DOLocationID',right_on='LocationID')
dropoff_location_dim['dropoff_borough']=dropoff_location_dim['Borough']
dropoff_location_dim['dropoff_zone']=dropoff_location_dim['Zone']
dropoff_location_dim=dropoff_location_dim[['dropoff_location_id','dropoff_borough','dropoff_zone']]
dropoff_location_dim.head()

,dropoff_location_id,dropoff_borough,dropoff_zone
0,0,Manhattan,Lenox Hill West
1,108,Manhattan,Lenox Hill West
2,134,Manhattan,Lenox Hill West
3,175,Manhattan,Lenox Hill West
4,199,Manhattan,Lenox Hill West


In [74]:
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
payment_type_dim = df[['payment_type']].reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)
payment_type_dim = payment_type_dim[['payment_type_id','payment_type','payment_type_name']]

In [75]:
fact_table = df.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
             .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
             .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
             .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
             .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id')\
             .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
             .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
             [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]

In [76]:
fact_table.head()

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,1,0,0,0,0,N,0,0,0,26.5,4.25,0.5,7.60,6.55,0.3,45.70
1,1,2,1,1,1,1,N,1,1,1,13.0,0.50,0.5,0.00,0.00,0.3,16.80
2,2,1,2,2,2,2,N,2,2,2,13.5,3.00,0.5,2.00,0.00,0.3,19.30
3,3,1,3,3,3,3,N,3,3,3,6.5,3.00,0.5,2.05,0.00,0.3,12.35
4,4,1,4,4,4,4,N,4,4,4,12.5,3.00,0.5,2.50,0.00,0.3,18.80
